In [29]:
!pip install -U sentence-transformers
!pip install torch
!pip install json
!pip install transformers

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [30]:
from sentence_transformers import SentenceTransformer, util
import json
import torch
import os
import zipfile

In [31]:
def unzip_file(zip_path, extract_to):
    # Ensure the extraction directory exists
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Open the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents into the specified directory
        zip_ref.extractall(extract_to)

zip_path = './tuned_bi_directory/tuned_bi.zip'
extract_to = 'tuned_bi_directory'
unzip_file(zip_path, extract_to)

In [32]:
import json

def format_json(file_path):
    # Read the JSON data from the file
    with open(file_path, 'r', encoding='utf-8') as infile:
        data = json.load(infile)

    # Write the formatted JSON data back to the same file
    with open(file_path, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, indent=4)


In [39]:
import json

# Takes Test qrel and then makes it into new json of just id -> query_text
def match_and_update_text(qrels_path, topics_path, output_path):
    with open(qrels_path, 'r', encoding='utf-8') as qrels_file:
        qrels_data = json.load(qrels_file)

    with open(topics_path, 'r', encoding='utf-8') as topics_file:
        topics_data = json.load(topics_file)

    topics_dict = {str(topic['Id']): topic['Title'] + " " + topic['Body'] + " " + " ".join(topic['Tags']) for topic in topics_data}

    updated_qrels_data = {}

    for query_id in qrels_data.keys():
        if query_id in topics_dict:
            updated_qrels_data[query_id] = {'query_text': topics_dict[query_id]}

    with open(output_path, 'w', encoding='utf-8') as output_file:
        json.dump(updated_qrels_data, output_file, indent=4)

def prepare_all_topics_with_query_text(topics_path, output_path):
    with open(topics_path, 'r', encoding='utf-8') as topics_file:
        topics_data = json.load(topics_file)

    # Prepare a dictionary with all topics, where each key is a topic ID
    all_topics_data = {
        str(topic['Id']): {'query_text': topic['Title'] + " " + topic['Body'] + " " + " ".join(topic['Tags'])}
        for topic in topics_data
    }

    with open(output_path, 'w', encoding='utf-8') as output_file:
        json.dump(all_topics_data, output_file, indent=4)


# Qrels only for getting test data
#qrels_path = 'updated_test_qrels.json'
topics_path = 'topics_2.json'
#output_path = 'updated_test_qrels.json'
prepare_all_topics_with_query_text(topics_path, 'updated_topics.json')
#match_and_update_text(prepare_top, topics_path, output_path)

In [40]:
# Define a function to load JSON files
def load_json_file(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

# paths to the files
topic_filepath = 'topics_2.json'
answer_filepath = 'Answers.json'

In [41]:
import re
import bs4
def clean_topics_text(item):

    full_string = item['query_text']

    full_string = bs4.BeautifulSoup(full_string, 'html.parser').text
    full_string = re.sub(r'[^a-zA-Z\s]', ' ', full_string)

    return full_string

def clean_answers_text(item):
    full_string = bs4.BeautifulSoup(item['Text'], 'html.parser').text
    full_string = re.sub(r'[^a-zA-Z\s]', ' ', full_string)

    return full_string

topic_filepath = 'updated_topics.json'

# Clean em' up
topics = load_json_file(topic_filepath)
answers = load_json_file(answer_filepath)


cleaned_topics = [{'id': key, 'text': clean_topics_text(value)} for key,value in topics.items()]
cleaned_answers = [{'id': answer['Id'], 'text': clean_answers_text(answer)} for answer in answers]


In [42]:

model_path = 'tuned_bi_directory'

bi_encoder = SentenceTransformer(model_path, local_files_only=False)
topic_embeddings = bi_encoder.encode([topic['text'] for topic in cleaned_topics], convert_to_tensor=True)
answer_embeddings = bi_encoder.encode([answer['text'] for answer in cleaned_answers], convert_to_tensor=True)


In [43]:
torch.save(topic_embeddings, 'topic_embeddings.pt')
torch.save(answer_embeddings, 'answer_embeddings.pt')

# Optionally, save the cosine similarity matrix
cos_sim = util.cos_sim(topic_embeddings, answer_embeddings)
torch.save(cos_sim, 'cos_sim.pt')

In [44]:
# Step 1: Load the saved embeddings
topic_embeddings = torch.load('topic_embeddings.pt')
answer_embeddings = torch.load('answer_embeddings.pt')

# Step 2: Compute cosine similarity between topic embeddings and answer embeddings
cos_sim = util.cos_sim(topic_embeddings, answer_embeddings)

# Step 3: Get the top 100 results for each topic
top_k = 100
top_results = []

# Iterate over topics and find top 100 answers for each
for i, topic in enumerate(cleaned_topics):
    topic_id = topic['id']
    topic_text = topic['text']

    # Compute cosine similarity between the topic and all answers
    cos_sim = torch.nn.functional.cosine_similarity(topic_embeddings[i], answer_embeddings)

    # Get top_k answers
    top_k_values, top_k_indices = torch.topk(cos_sim, k=top_k)

    # Prepare the results for this topic
    topic_results = {
        "query_id": topic_id,
        "query_text": topic_text,
        "results": [
            {"answer_id": cleaned_answers[idx]['id'], "answer_text": cleaned_answers[idx]['text'], "score": val.item()}
            for idx, val in zip(top_k_indices, top_k_values)
        ]
    }
    top_results.append(topic_results)

# Save the top results to a file
with open('top_100_results.json', 'w') as f:
    json.dump(top_results, f, indent=4)


<ipython-input-44-35f5a9d3e1d6>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  topic_embeddings = torch.load('topic_embeddings.pt')
<ipython-input-44-35f5a9d3e1d6>:3: Futu